## Region selected  for the project
I decided to work with the map of the city of Zurich, Switzerland. For private and professional reasons, I will be relocating to Zurich and I figured it might be a good idea to get acquainted with the city I will be living in in the near future.
For this purpose an OSM file in XML format, representing the city of Zurich, was downloaded from http://www.openstreetmap.org.

## Get an overview about the OSM data 
After downloading the OSM file, I created a smaller OSM subset (zurich_sample.osm) that contains the main elements (node, way, relation). This file was manually pre-screened to get an idea how particular information is stored. Moreover, the smaller variant of the OSM file was used to check if any code (e.g. code for auditing) is working, before applying the code together with the original-size OSM file.
Information on the general structure of the OSM XML file format can be found on http://wiki.openstreetmap.org/wiki. 

In [ ]:
osm_file = "zurich.osm"
def get_element(osm_file, elements = ("node", "way", "relation")):
    treebuilder = ET.iterparse(osm_file, events = ("start","end"))
    # skip the the root element and the next 3 elements (bounds, meta, node)
    for i in range(3):
        _, skip = next(treebuilder)
    for event, element in treebuilder:
        if event == "end" and element.tag in elements:
            yield element
            skip.clear()
            
           
with open("zurich_sample.osm", "w") as file_out:
    file_out.write("<?xml version='1.0' encoding='UTF-8'?>\n")
    file_out.write("<osm version='0.6' generator='Overpass API'>\n")
    file_out.write("<note>The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.</note>\n")
    file_out.write("<meta osm_base='2017-05-15T16:51:17Z'/>\n")
    file_out.write("<bounds minlat='47.2734000' maxlon='8.8031000' minlon='8.2700000' maxlat='47.4815000'/>\n")
    
    for i, element in enumerate(get_element(osm_file)):
        if i%4000 == 0:
            file_out.write(ET.tostring(element, encoding='utf-8'))
            
    file_out.write("</osm>\n")  

## What to audit?
I selected following element attributes of the OSM file for auditing and cleaning:
1. id, uid and version (audit_id_version.py)
2. lat and lon (audit_coordinates.py)
3. timestamp (audit_timestamp.py)
4. addr:street (audit_street.py)
5. addr:housenumber (audit_housenumber.py)
6. addr:city (audit_city.py, crossaudit_city_postcode.py)
7. addr:postcode (audit_postcodes.py, crossaudit_city_postcode.py)


Scripts used for auditing are given in parentheses. For more information about the scripts used for data auditing, cleaning and processing, please refer to the readme file.

## Audit element id, uid and version
Id's and user Id's are either stored as attributes "id" and "uid" in all primary elements or they are stored as the attribute "ref" in the second level elements nd (child of first level element way) and member (child of first level element relation). The attribute "version" is stored in all first level elements.
Valid "id", "uid" and "version" attribute values must be convertible to integer type. 

### Results
no irregularities found

## Audit geographical coordinates (latitude and longitude)
The attributes "lat" and "lon" represent geographical coordinates and are stored in the first level element node. Valid coordinates must be convertible to float type but not integer type

### Results
no irregularities found

## Audit timestamp 
The attribute timestamp refers to a representation of time and date and is stored in all three first level elements.
According to the International Standard for the representation of dates and times, valid time formats are (which can be querried with SQLite):
1. YYYY-MM-DD hh:mm:ss
2. YYYY-MM-DDThh:mm:ssZ

Y: Year, M: Month, D: Day, h: Hour, m: Minute, s: second
T seperated date and time and Z designates UTC (Coordinated Universal Time) time

- please refer to https://www.w3.org/TR/NOTE-datetime for more information 

### Results
no irregularities found

## Audit street names

Street name attributes are stored as key-value pairs with key = "addr:street" tagging the attribute as an information container for a street name and value = "any name" defining the actual name of the street. "addr:street" is stored in second level elements tag.  

Unfortunately there is no concise terminology or convention for naming streets in german-speaking countries (e.g. Switzerland). Sometimes they can be written in two words, where one Noun is joined with a "typical" street category (Strasse -street-, Platz -square-, Allee -avenue-, Weg -way-), as in "Mainzer Strasse", which implies the meaning "in honor of" (street named in honor of the City Mainz). However, two words can be combined to name a street, without any of the words describing a typical street category (e.g, "Oberere Weid", "Hinter Zünen") or even more than two words can be used to name a street (e.g, "An der Halde"). In both cases -without street category extension- the name simply has an inherent geographic description for that particular street. E.g, "Oberere Weid" means that the street is located next to a high-altitude meadow or atop a mountain/hill, as in "Am Berg". As if this isn't complicated enough, basically the same street name can be written in multiple variations depending on regional specifications. E.g, "Obere Weid", "Ober Weid", "Obere Weide", "Obere Weiden" are all valid street names in Switzerland/Germany. So with respect to the latter example, you can't be sure that "Weid"/"Weiden" is a misspelling of "Weide"!
Additionally, street names can consist of one-word trivial names that are inherently ambiguous. E.g., "Vorstadt" is a valid street name in Zurich but the word literally translates to "suburb". Moreover, each of the above mentioned two-word variants are existent as one-word combinations, such as "Holzweid" or "Müssmattstrasse". In fact, one-word street names are the most common variant!!  

As such, the most significant issue in terms of identifying any wrong spelling or abbreviations  in street names, is the lack of standardized naming pattern, e.g with street types as distinct words separated from the rest of the name, which is the standard for UK/ american street names (e.g "7th avenue"). On the one hand, this fact makes it necessary to use basic regular expression checks to screen for misspelling or abbreviations for street types, which on the other hand, impedes extraction of different types of abbreviations or misspelling, as for instance using "st" as a regex check to identify "-str"/"-strrasse"/"-stasse", without mostly and wrongly extracting valid street names, such as "Kirchenstiege" (by using st in a regular expression check)!! This is even more of an issue for street types having, for instance "Allee" or "Weg"  in their names, as the combination of vowel-consonant/consonant-vowel, if using "al or we" as a regular expression check, is vastly present in valid street names ("Waldgass", "Waltikon" and many many more would be returned as invalid). The bigger the dataset, the more apparently invalid street names will be returned, which at some point makes automated auditing obsolete!!


### Strategy
So I developed following strategy to identify most of invalid street name entries without returning too many false negatives and, thus, making the analysis very cumbersome.

1. For each of the expected street types, check for word-character insertions (e.g, insertion of "a" in "Plaatz"). Including insertions of characters not present by default in the name (e.g "e" in "Pleatz") would require a much more sophisticated approach, which I am not yet capable of implementing.

2. Check for abbreviations or deletion.  
-For each of the names in the expected street types set, use the first two characters to define a regular expression  (e.g, "st" to find Müssmattstr. or Müssmattstrase).  
-allow 3 additional word-characters to follow the two-letter regular expression for all expected street types but "strasse". For "strasse" allow up to 4 additional word-characters (simply because "strasse" is the string with most leters).
-restrict the regular expression check to search for a match at the end of a string

I make the assumption that insertions include only characters from which the street type name is constructed of (e.g "p,l,a,t,z" in "Plaatz" with "a" insertion), simply by pressing a key twice by mistake. Separating auditing for misspelling by insertion and deletion/abbreviation,  allows for a more stringent search of deletions, because I can reduce the number of word-characters following the two-letter regular expression. As such, I assume to overall minimize the number of false positives (e.g regex check using "st" would return "Rostlaubenweg" -valid name-, if more than 4 additional word-characters are allowed in the expression) without missing too many true positives (regex matching misspelling of street type).



### Results

**misspelling by insertion** as in "Römerstrassse" ("s" insertion in strasse)

**misspelling by deletion** as in "Hofwiesenstrsse" ("a" deletion in strasse)

**use of dialect** as in "Im Chlösterli" or "Gass" (in official language "Im Kloster" or "Gasse")
this is not wrong per se, however, if including validation of street names against a reference database, it might be a problem. I will go into more detail in the section  "Strategy for correcting street names, city name and postcodes"

**abbreviations** as in "Brunnackerstr" (strasse instead of str)

**digits in street names** as in "zuercherstrasse 95" or "Schiffbaustrasse 9b" (housenumber as part of the name)

**name consisting of digit only** (e.g, "6")

**abbreviations and digits in street name** as in "Alte Bahnhofstr. 22"

The follow up data cleaning of street names includes correction of insertion, deletion and abbreviations in street types into the expected naming pattern. Digits will be removed from street names and street names consisting of digits only, will be excluded in the process of setting up the database. For this purpose, the function street_clean() was used (see osm_cleaning.py script). For more details, please refer to the section "Strategy for correcting street names, city name and postcodes".

## Audit housenumber
Housenumbers are also stored in second level elements tag as key-value pairs with key = "addr:housenumber".

### results first iteration
Initially, I defined valid housenumbers to consist of digits only, or digits followed by one word character. The respective audit function returned all values that don't match a valid housenumber (via regular expression check).  
This first check returned many entries consisting of multiple housenumbers (separated by different non-word characters; e.g 53/54 and optionally including word-characters; e.g 9a-9c) stored as one value. However, based on openstreetmap wiki, multiple housenumbers are expected to occur. 

### results second iteration
As such, I conducted a second iteration using a different regular expression that matches single or multiple housenumbers, possibly extended by word characters (e.g, 9, 9a, 9 a, 9-9, 9-9b, 9a-9b). In this approach, I assign all values that **do not match** the regular expression as being an invalid housenumber!
Following invalid entries were returned after the second iteration step: only word characters (e.g "A"), word character followed by digits (A4812), street name with/without housenumber (e.g "Im Chies 14", "144 Im Hof", "Sportschützen Uster").

Of all the invalid housenumber values, only those having the street name and the housenumber can be corrected by simply stripping the street name. The other invalid values will be will be excluded in the process of setting up the database. A4812 has the format of swiss highways (A followed by a number), however the number is too large.

## Audit City
Similar to street names, the city name attribute is stored in the second level element tag as a key-value pair with key = "addr:city".

I expected all city names (values for the attribute "addr:city" in second level "tag" element) being confined to "Zurich". However, I wanted to get an overview if different spelling variants for "Zurich" were used. This is very common for german words that have muted vowels (Umlaut), such as "ä,ü,ö". Expected variants include "Zürich" (muted vowel  "ü)", "Zuerich" (ue substituted for muted vowel ü) and "Zurich" (common variant in english, which simply drops the diacritical marks).  
However, after having a sneak peak at the smaller Zurich OSM sample, I noticed for some second level tag elements to have city names other than Zurich (e.g "Dielsdorf"). Therefore, I used a function to return a set for all Zurich variants, as well as a set for all other names. 


### Results first iteration
All of the aforementioned spelling variants of Zurich were found. Additionally, entries with the city name of Zurich in combination with a city district were present (e.g "Zurich-Altstetten"). Additionally, names entirely composed of digits (e.g 8002 and 50; likely postcodes and housenumbers) were also present. However, much more cities other than Zurich were returned, some of which having abbreviations (e.g "Affoltern a.A.") or two-letter extensions (e.g. "Aesch (ZH))" as part of the name.  
It seems that the Zurich city OSM file includes entries that actually describe the Canton of Zurich. Cantons are the member states of the country of Switzerland and the Canton of Zurich comprises of 12 districts to which the city of Zurich as well as the aforementioned Dieseldorf belong.  
Instead of screening the entire long list returned by audit_city() function, I decided to include a second iteration and systematically check for entries containing particular non-word characters, as these seem to be present in city names that have abbreviations and word extensions.

### Results second iteration
More abbreviation variants were found (e.g "Oberwil b. Nürensdorf", "Wettswil a. Albis"). Additionally, two-letter extensions in different variations (e.g "Buchs (ZH)", "Buchs ZH", "Winterberg/ZH"), as well as word extensions in different variations (e.g "Riedikon (Uster)", "Forch (Aesch, Maur)", "Aathal-Seegräben", "Grafstal/Kempttal") were found. Furthermore, 4 cities with state extension other than state (Canton) of Zurich  were found, e.g "Muri (AG)", a city located in the state of Aargau. Finally, different spelling styles for the same city name were found (e.g "Aathal - Seegräben" and "Aathal-Seegräben", "Uitikon-Waldegg" and "Uitikon Waldegg")

Extensions to city names either provide the affiliation of a village/city to a certain state in switzerland (in case of two-letter extensions) or to a certain municipality (e.g "Oberwil"; city "Oberwil" in municipality "Nürensdorf" versus municipality "Cham" ) or to both, municipality and state ("Oberwil" in municipality "Nürensdorf" in state "Zurich" versus "Oberwil" in municipality "Cham" in state "Zug").  
However, it is very difficult to check for valid naming pattern for names with word extensions without having some kind of reference. The reason is that for most names, it is unclear whether the first or the second word describes the city name or the municipality name, or maybe neither of both, or if the same city name with and without an extension actually refers to the same city or to different cities located in different municipalities/states (e.g "Ottikon" vs. "Ottikon" (Gossau)"). As such, a cleaning strategy that for example would only consider the first word in a name as the city name, would return many false positives. 

So I decided to correct abbreviations, different spelling styles for the same city name, digits as name, and correct two-letter extensions to have one consistent form. City names with state affiliation other than state of Zurich will be excluded. Knowledge about the meaning of abbreviations is based on experience in german language or from cognate city names without abbreviations.

## Audit postcode
Postcodes are stored in second level elements tag as key-value pairs with key = "addr:postcode".

Valid Zurich city postcodes are supposed to be 4-digit numbers starting with 8 and being in the range 8001-8064. Based on the results from auditing city names (city names other than Zurich city), I expected to find postcodes that are not conform with specifications for valid Zurich city postcodes. 

### Results
Accordingly, 4-digit Postcodes starting with 3,4,5,6 and 9 were found, as well as postcodes consisting of word characters (e.g "q") were found. I decided to exclude those postcodes consisting of word-characters only and to keep all other postcodes 

Expected Postcodes were obtained from http://www.plz-suche.org/zuerich-ch7e45 and can be found in audit_postcodes.py.

## Cross-Audit postcode/City
Bearing in mind that unexpected city names and postcodes are present in the OSM file, I furthermore assumed wrong combinations of city name and postcode, either unexpected postcodes referring to the city of Zurich or unexpected city name referring to expected postcodes for the city of Zurich.  

### Results
Quite a lot of invalid city name-postcode combinations were found. E.g, city name "Baden" refering to 8 different postcodes, all of which are expected for the city of Zurich, or unexpected postcode 5400 referring to the city of Zurich. Additionally, postcodes starting with 8 but not belonging to the expected set of Zurich postcodes (e.g, postcode 8330 in combination with Zurich as the city).

## Strategy for reviewing the validity of street names, city name and postcodes
To me, it was very surprising to find so many wrong city names and postcodes present in the OSM file that is supposed to contain information exclusively about the city of Zurich. As already noticed, many of those wrong entries simply refer to the state of Zurich instead of being restricted to the city of Zurich! Based on this explicit mistakes, I strongly assume that many street names are included that actually don't belong to the street set of the city of Zurich!! For first level elements (node, way, relation) having tags specifying street names, city names and postcodes, respectively, it is basically unclear which entries are actually right or wrong in terms of validity (just the postcode, or both postcode and city, or is the street located outside the city boundaries of Zurich city?). 

Because the OSM file contains thousands of wrong entries, I decided to set up a "gold standard" with valid street names and corresponding postcodes for the city of Zurich, which can be used to audit validity of OSM address tags. 

### Define the gold standard to validate address specifications in OSM file
The goal was to create a reference dataset that contains all street names and the corresponding postcodes. Since Zurich is divided in 11 districts (e.g "Kreis-7"), each in turn consisting of different quarters (e.g quarters "Hottingen" and "Witikon" are located in district "Kreis-7"), I decided to include information about the district and quarter affiliation in the reference dataset. So, the final csv files and SQL database, respectively, will be extended by address information on quarter and district affiliation.  

I used http://www.streetdir.ch/CH/Zurich/Zurich/Strassenverzeichnis to mine data containing the relevant information. The browser developer tool was used to determine the valid method for making successful HTTP requests (GET method) and to screen how the relevant data (street name, quarter, district, postcode) is stored in the HTML documents. 

The above mentioned URL retrieves the html document that contains all streetnames with initial letter A-F. By extending the URL with query parameters "G", "L" or "S", you will retrieve the html documents containing street names with initial letter G-K, L-R and S-Z, respectively. Each street is stored as a hyperlink, which will be used to eventually fetch the street name, quarter and district affiliation and postcode.
**The function query_site() will be used to make HTTP requests (see code in section below)**

In the cognate HTML files, relevant street information is stored in hyperlink tags "a" within "table" tags. Each hyperlink tag contains a "class" and "href" attribute. The "class" attribute value ("DirectoryStreetLink") can be used to select (via BeautifulSoup) all street names and relevant information ("href" value), respectively, to retrieve cognate quarter, districts and postcodes.
**The function get_street_information() makes use of the query_site() function to make all HTTP requests and to save the data as csv file (see code in section below)** 

### Rules for auditing validity and making updates
**If OSM street name is present in reference dataset**
1. if city name is Zurich, update postcode, district and quarter according to reference data for the cognate street name

2. if postcode is expected for Zurich, update city name, district and quarter according to reference data for the cognate street name

3. if first level element has only a street tag (no tag for city, postcode), create tags for city name, postcode, district and quarter according to reference data for the cognate street name.

matching either postcode or city name together with street name is required to discriminate same street names belonging to different cities 

**If OSM street name is not present in reference dataset**
1. if city tag value is Zurich, update value to "Zurich municipality"

2. if first level element has no city tag, create city tag with "Zurich municipality" as value

3. if city tag value is not Zurich, keep city name

In [716]:
import csv
import requests
from bs4 import BeautifulSoup
import codecs



# from Class
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

url_BASIC = "http://www.streetdir.ch"
url_STREETDIRECTORY = url_BASIC + "/CH/Zurich/Zurich/Strassenverzeichnis"
params = (None, "G", "L", "S")


def query_site(url, params):
    '''
    main function to make queries
         
    params:  query parameter to retrive the correct html file (streetnames with particular starting word characters).
             G, L, S, None are valid params (If params = None provided, query will be performed without a query
             parameter, which will return streetnames A-F). 
             G = street names G-K; L = street names L-R; S = street names S-Z
    url: url string         
    
    '''
    params_dict = {"char" : params}
    r = requests.get(url,params_dict)
    return r.text

def get_street_information(url_BASIC,url_STREETDIRECTORY, params):
    '''
    uses the query_site function to fetch names, zipcodes, district and quarter affiliation for all streets in zurich.
    
    url_STREETDIRECTORY (http://www.streetdir.ch/CH/Zurich/Zurich/Strassenverzeichnis) -can be extended by query 
    parameter "char"- lists all street names for the city of Zurich. For each street a relative URL path will be 
    extracted that is joined with url_BASIC and provides a new absolute URL, which is used to scrape (via query_site()
    and BeautifulSoup) the name, quarter and district affiliation and postcode.
    
    
    Data will be written as csv file 
    '''
    
    
    with codecs.open("street_names_zipcodes_zurich", "w") as file_out:
        fieldnames = ["district", "quarter", "street", "zipcode"]
        writer = UnicodeDictWriter(file_out, fieldnames=fieldnames)
        writer.writeheader()
    
        for query_param in params:
            data = query_site(url_STREETDIRECTORY, query_param)
            soup = BeautifulSoup(data,"lxml")
    
            for item in soup.findAll("a", attrs={'class':'DirectoryStreetLink'}):
                street_PATH = item.get("href")
                data = query_site(url_BASIC + street_PATH, None)
                soup_within = BeautifulSoup(data,"lxml")
                
                street = soup_within.find("span", attrs={"id":"yah5"}).text
                quarter = soup_within.find("span", attrs={"id":"yah4"}).text
                zipcode = soup_within.find("meta", attrs={"name":"keywords"})["content"].split(",")[-1]
                district = soup_within.find("span", attrs={"id":"yah3"}).text
                
                writer.writerow({"district":district, "quarter":quarter, "street":street, "zipcode":zipcode})
                
        
    
    
get_street_information(url_BASIC,url_STREETDIRECTORY,params)    

## Audit the reference dataset
Similar to the OSM data, the reference dataset requires a screening to identify potentially invalid entries. 

### First iteration
Following results were recorded. Districts are either specified by numbers (e.g, "Kreis-9"), or by trivial names (e.g "Wiedikon"). Both are correct, but for the purpose of consistency trivial names will be substituted for the corresponding numbered variants (e.g "Wiedikon" substituted for "Kreis-3"; data obtained from https://en.wikipedia.org/wiki/Subdivisions_of_Z%C3%BCrich). The code below was used to update districts
Some of the postcodes are not valid postcodes for the city of Zurich (e.g, 8122, 8142) and some postcodes have a wrong format by starting with 0 (e.g, 08049). No irregularities in street names were found. 
Based on the irregularities found, I suspect invalid combinations of postcodes and quarters/districts or quarters and districts. 

auditing was conducted by using audit_reference_data() function present in audit_reference.py.

In [81]:
mapping_districts = {"Industriequartier" : "Kreis 5",
                     "Wiedikon" : "Kreis 3",
                     "Schwamendingen" : "Kreis 12",
                     "Aussersihl" : "Kreis 4",
                     "Riesbach" : "Kreis 8",
                     "Altstadt" : "Kreis 1"}

def update_districts(entry):
    '''
    use mapping_districts to update districts
    '''
    if entry in mapping_districts:
        return mapping_districts[entry]
    else:
        return entry
    
ref_data = pd.read_csv("street_names_zipcodes_zurich", dtype=str)    

**update district naming** 

In [82]:
ref_data["district"] = ref_data["district"].apply(update_districts) 

### Second iteration
Consequently, I defined two sets of expected combinations of districts and quarters, as well as quarters and postcodes (data obtained from http://www.plz-suche.org/zuerich-ch7e45, https://en.wikipedia.org/wiki/Subdivisions_of_Z%C3%BCrich; sets can be found in audit_reference.py). After updating district names to numbered versions, no invalid combinations between quarters and districts were found. However, quarters with wrong postcodes were found -additionally to the expected invalid combinations for postcodes starting with 0 and those postcodes invalid for the city of Zurich- (e.g, quarter "Witikon" with postcode 8032, which is the postcode for quarter "Hirslanden"). 
Following code was used to correct postcodes. An updated version of the reference dataset ("street_names_zipcodes_zurich_update") was generated, which will be used as the "gold standard" to validate address specifications of the zurich OSM file.

auditing was conducted by using audit_district_quarter_crossref() and audit_quarter_postcode_crossref() functions present in audit_reference.py.

**update postcodes**

In [83]:
from audit_reference import*

def update_postcodes(entry):
    '''
    use expected_quarters_postcodes dictionary from audit_reference.py to correct postcodes
    '''
    return expected_quarters_postcodes[entry["quarter"]]   


ref_data["zipcode"] = ref_data.apply(update_postcodes, axis=1)

### Third iteration
I realized that I need to check whether the street names in the reference dataset are unique. This is important to adequately adapt the implementation of validating and updating the Zurich OSM file by using the reference dataset (see section "Updated rules for auditing validity" below). 
Indeed, some street names were present multiple times in the reference dataset (e.g, 2x "Flurstrasse"). I included "district", "street" and "postcode" information to verify only duplicates having the same "postcode" and "district". These will be removed from the reference dataset, whereas duplicate street names from different districts will remain in the reference dataset, as for those no further -automized- validation is possible and same street names for different districts exist, respectively. 

**remove street name duplicates and write updated reference dataset**

In [84]:
ref_data = ref_data.drop(ref_data[ref_data[["district","street","zipcode"]].duplicated()].index.values)
ref_data.to_csv("street_names_zipcodes_zurich_update", index = False)

### Updated rules for auditing validity and making updates
**If OSM street name is present multiple times in reference dataset**
1. If the cognate OSM postcode (from same first level element) has a match in the reference dataset that corresponds to the street name, update city, district and quarter according to reference data for the cognate street name

No updates If the cognate OSM postcode does not match the reference dataset postcode that corresponds to the OSM street name, or if no OSM postcode tag is present.

## Ultimate procedure for cleaning street names, city name and postcodes
First stage of cleaning includes the correction of basic irregularities, e.g correction of abbreviations, misspelling in street names, removal of no-digit postcodes or consistent spelling for Zurich, etc. The relevant functions for first stage cleaning are included in osm_cleaning.py.
Following first stage, the second stage will include the revised reference dataset ("street_names_zipcodes_zurich_update") for validation of relevant values (street name, city name, postcode) and making appropriate updates (include district and quarter affiliation). Validation and update rules are described in the sections "updated -/ rules for auditing validity and making updates" in more detail. data.py implements both cleaning steps and eventually creates csv files containing the relevant and cleand data points from the Zurich OSM file (for more information refer to data.py).

## Setting up the database
The SQL database (zurichOSM.db) was created via command line using sqlite3. For each csv file returned by data.py, a database table was created with standard SQL table schema (for more information, please refer to www.sqlite.org) using following command. 

If appropriate, PRIMARY KEY or FOREIGN KEY constraints were specified. After creation of a table, following command was used to import the corresponding csv file into the table.

Be aware that for csv files with header, the header will be imported into the table when using sqlite3 command line!!
Delete header row from table. E.g, following command will delete the row with id as value for the column id from relations table. Header rows have been deleted for all tables in the zurichOSM.db.

## Overview statistics

all queries conducted via command line using sqlite3
### Number of nodes

**1993633**

### Number of ways

**314124**

### Number of relations

**5046**

### Number of unique users

**2356**

### Earliest and latest contribution

**2006-05-05** and **2017-05-15**

## Additional statistics

### Min, max and average contributions per year and total contributions for top 10 users

### Contribution of top 1%, 5% and 10% of users
24, 118 and 236 users with highest total contributions depict the top 1-,5- and 10% of users, respectively. Use this numbers for the LIMIT command in the code below to obtain the respective perentages.

**Top 1%** made 72.55% of total contributions  
**Top 5%** made 93.43% of total contributions  
**Top 10%** made 96.93% of total contributions  

Only 5% of total users made almost 95% of the total dataset (contribution reaches a plateau from including the top 5%) !!

### Number of restaurants, cafes, bars, pubs and top 20 cuisines

**2577** places serving food and beverage

### Number of restaurants, cafes, bars, pubs and top 20 cuisines in Zürich

**736** localities in Zürich serving food and beverage

Because it is not really clear what kind of cuisine "coffe_shop" is supposed to be, I will include cuisine ranked 21st in the top 20 and ignore "coffe_shop".

The OSM data contains information on cuisine type for 324 restaurants in Zürich in total (without "coffe_shop"; use the above query without grouping by cuisine value). Italian(including pizza), asian(including chinese, thai, japanese and sushi)  and regional (including swiss) cuisines are the most prominent.

### Number of restaurants, cafes, bars, pubs and top 3 cuisines per district 

As expected, most food and beverage places are located in district "Kreis 1", which is the city center/old town

Again, because of the ambiguous cuisine type "coffe_shop", I queried the top 4 cuisines, excluded "coffe_shop" if present in top 3, and included the cuisine ranked one position behind "coffe_shop". This is true for district 1, district 6 and district 11 (For the sake of clarity, manually filtered results for top 3 cuisines are shown).

The data on cuisine types per district matches the ranking obtained for the city (see section above), with italian, asian or regional cuisines being the most predominant. Only in district 5 and 12, indian and bagels, respectively, are the top ranked cuisines.

### Number of facilities related to education, healthcare and public service for each district
This includes a personal selection of "nice to have" nearby

Highest and lowest counts are obtained for district 1 and 4/5, whereas similar counts are obtained for all other districts (please refere to the section Possibilities for improvements for more discussion on district-specific results).

### Number of facilities related to leisure, sports and entertainment for each district
Again, a selection of sports activities and entertainment for which I would like to have the appropriate infrastructure nearby. 

District 1 has the highest number of relevant facilitiest, whereas all other districts exibit rather lower counts (please refere to the section Possibilities for improvements for more discussion on district-specific results).

### Number of shops for each district
This is a selection of shops that I would consider nice to have nearby. 

district   count     district   count

Kreis 1  | 38        Kreis 5  | 19
Kreis 11 | 38        Kreis 3  | 18
Kreis 2  | 30        Kreis 8  | 18
Kreis 9  | 28        Kreis 10 | 16
Kreis 6  | 23        Kreis 7  | 11
Kreis 4  | 20        Kreis 12 | 4

As expected, more or less similar counts were obtained for all districts but for district 7 and 12. The lower counts are probably due to missing  information in the OSM dataset and don't represent true numbers for shops (please refer to the section Possibilities for improvements for more discussion on district-specific results).

## Possibilities for improvement of the dataset or improvement of the analysis 
**increasing the number of contributions and quantity of relevant information**  
Only for 324 of the total 736 entries depicting food and beverage localities in Zurich, the relevant information on the cuisine type is present in the dataset (534 total entries and 317 including cuisine, if restricted to only restaurant; queries not shown).  
Maybe the restaurants themselves can contribute to encourage the user community to participate in updating the OSM dataset. I am thinking of restaurants offering voucher for meals/drinks to anyone who makes updates on the dataset that would provide the most relevant information about the cognate restaurant (street name, postcode, district and quarter affiliation, opening hours, cuisine, average price for meals, etc.). This would be attractive especially for people living nearby a restaurant (or at least in the same quarter/district), as the benefit of getting free meals/discounts could prevail the effort for conducting an update (assuming basic computer knowledge). The restaurants could try to stimulate overall contributions by offering really valuable benefits (expensive meals, meals for two people, etc.) as the overall costs would still be very low, in contrast to extremely high marketing value.  

In general, there is a discrepancy between the total counts for particular map features for the city of Zürich and the total counts for the same map feature but including information on district affiliation. For example, there are 263 "shops" that also contain information on district affiliation (use query in the section above without grouping), whereas a total of 1199 "shops" in Zürich are present in the dataset (see code below). So either the relevant XML elements have no street tags at all, or the combination of street tags/city tags and street tags/postcodes tags, respectively, don't match the reference dataset, which in both cases prevents to include information on district affiliation (for more details see section "Strategy for correcting street names, city name and postcodes").

Maybe the local government could support a general benefit/voucher-based system in context with creating/updating OSM data, maybe even in cooperation with regional facilities (e.g cinemas, etc.), e.g for every 50-100 updates, you would get a discount in the local cinema/theatre. Again, this might have a big marketing impact without creating too high costs for such a program. 

The main issue would be to ensure that user contribution is somehow officially verifiable (e.g, proof for eligibility for any benefits/discounts) and that concomitantly the manipulation of OSM data matches certain quality standards (you don't want users to simply do nonsense contributions to be eligible for benefits/discounts). For the second point, some kind of "audit automation (maybe similar scripts as used in this project)" that automatically screen every new entry would be required in the backend.

**improving the accuracy of contributions**  
Only 45% of all city tags actually correspond to the city of Zürich (see query below)!!! So the majority of entries including information about city affiliation actually depict Zurich municipality. The reason, on the one hand, is that the Export function used to download the XML file from the openstreetmap homepage is using wrong default values for the city boundaries of Zürich city (minlat='47.2734000' maxlon='8.8031000' minlon='8.2700000' maxlat='47.4815000'). Thats an important detail one cannot expect to be aware of **"You don't know what you don't know"**. I checked the border coordinates manually, only because of the large number of wrong city tags and postcode tags exposed from auditing the data. The approximate correct city coordinates are: minlat='47.3381', maxlon='8.6270', minlon='8.4481', maxlat='47.4351'.

However, there are 1055 nodes/tags that specify city as Zürich despite coordinates outside the boundaries of Zürich city (see query below). So I am thinking of constraints on coordinates -at least for major cities- in the backend. Each time user violate these predefined constraints with a wrong entry, the system should return a warning message stating that the respective coordinates might be wrong. On the other hand, including such constraints in data auditing, would be easy to implement in the underlying analysis. However, there should be at least an explicit warning message on ambiguous coordinates when downloading XML data from openstreetmap to remind users.

**improving quality of contributions**  
Dialect used for naming is present in the dataset (see query and result below) and this is especially a problem if you want to include a reference dataset for auditing/cleaning (for more information see auditing street names and strategy for correcting street names). Either you will fail to match with the reference data ("Chlösterlistrasse" in dialect refers actually to "Klosterstrasse") or some names might even be falsely identified as mistake-containing entries (like for the street type "gasse", which is written "gass" in dialect).
Actually, this problem can only be tackled by raising the awareness of users. Maybe, for each second level tag a mandatory key:value pair could be introduced that requires the user to specify whether the official language or a dialect has been used. This would still allow to use either of both, however, for third parties conducting data auditing/cleaning, this "language-version" could be of much help.  
Nevertheless, the big problem is that no objective control could be installed for what is actually assigned to the aforementioned key:value pairs. Consequently, users could make wrong specifications on purpose (e.g, specifying official language when using dialect instead).

## Conclusion
Screening for mistakes, , abbreviations, language variants in street names turned out to be more difficult than expected. The reason is that german street names don't follow a distinct naming pattern, including street types as independent words within the name. Instead a large number of streets has arbitrary names. Especially with growing size of data, the number of false negatives is increasing, which at some point will compromise the developed strategy, because false negatives will prevail.  
The other major issue was the presence of data points that actually don't belong to the selected dataset (here, the city Zurich in Switzerland). However, once being aware of this, one can easily implement a strategy to either select and mark those data points, or even exclude them from the final database.

## References used for the project

General Information on OSM data  
http://wiki.openstreetmap.org/wiki

Source to mine data for setting up the reference dataset for Zurich city  
http://www.streetdir.ch/CH/Zurich/Zurich/Strassenverzeichnis

Information about expected combinations of districts and quarters, as well as quarters and postcodes  
http://www.plz-suche.org/zuerich-ch7e45, 
https://en.wikipedia.org/wiki/Subdivisions_of_Z%C3%BCrich